In [15]:
import json
import pathlib
from pprint import pprint
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import re
from urllib.parse import urljoin
import sqlite3

In [2]:
json_file = 'pkgs.json'
db_file = 'pkgs.db'